In [10]:
import time
from api_secrets import *
from wiliot_api.edge.edge import BridgeAction
from wiliot_deployment_tools.api.extended_api import ExtendedEdgeClient, ExtendedPlatformClient, GatewayAction
from wiliot_deployment_tools.internal.Archive.firmware_update.firmware_update import FirmwareUpdate
from wiliot_deployment_tools.internal.Archive.power_mgmt.power_mgmt import PowerManagementClient
GW = 'GW864292045293444'
BRGS = ['7BA7DD8E304A', '0272A2B3B3E6',
        '639186EF4881', 'E465A89A65BE']
e = ExtendedEdgeClient(WH_EDGE, WH_OWNERID, 'prod')
p = ExtendedPlatformClient(WH_ASSET, WH_OWNERID, 'prod')
fw = FirmwareUpdate(WH_EDGE, WH_OWNERID, 'prod')
pm = PowerManagementClient(WH_EDGE, WH_OWNERID, 'prod')
print(e.get_gateway(GW))
# e.change_brg_config(BRGS, {'sub1GhzFrequency': 919100})

{'gatewayId': 'ERM864292045293444', 'gatewayType': 'lte', 'status': 'active', 'reportedConf': {'interfaceChipSwVersion': '3.10.34', 'bleChipSwVersion': '2.0.0', 'version': 'erm_3.10.34', 'apiVersion': '200', 'additional': {'gnssAlwaysOn': False, 'gnssMaxDuration': 250, 'bluetoothAlwaysOn': True, 'geoLocationRefreshTime': 60}}, 'desiredConf': {'interfaceChipSwVersion': '3.10.34', 'bleChipSwVersion': '2.0.0', 'version': 'erm_3.10.34', 'apiVersion': '200', 'additional': {'gnssAlwaysOn': False, 'gnssMaxDuration': 250, 'bluetoothAlwaysOn': True, 'geoLocationRefreshTime': 60}}, 'gatewayInfo': {'Cache': 'Pixels=9/128, Meta=5/256', 'Ignition switch state': '0', 'Battery voltage': '3.6', 'Cellular signal': '53A3:00A8AC15@-80 (9460)', 'ICCID': '89972010122060332131', 'Time since last boot': '0:00:00:40', 'ERM firmware version': '3.10.34', 'Cellular network state': 'Home', 'Hardware name': 'StarLink-W1 GWIFI2e32 AG', 'IMEI': '864292045293444', 'Cellular MNC-MCC': '425001', 'Cellular network': 'Pa

#### GW Config Change (All parameters available) + Timing

In [9]:
params = e.get_gateway(GW)['desiredConf']['additional']
new_params = {'gnssAlwaysOn': False, 'gnssMaxDuration': 250, 'bluetoothAlwaysOn': True, 'geoLocationRefreshTime': 60}
start = time.perf_counter()
e.change_gw_config([GW], new_params)
end = time.perf_counter()
print(start, end, (end-start))
start = time.perf_counter()
e.change_gw_config([GW], params)
end = time.perf_counter()
print(start, end, (end-start))



2023-05-21 12:07:28,555 - root - INFO - Configuration sent to ['ERM864292045293444']
2023-05-21 12:07:28,998 - root - INFO - ['ERM864292045293444'] updated to desired configuration:
2023-05-21 12:07:28,999 - root - INFO - ("{'gnssAlwaysOn': False, 'gnssMaxDuration': 250, 'bluetoothAlwaysOn': True, "
 "'geoLocationRefreshTime': 60}")


183.789950584 184.769044209 0.9790936249999902


2023-05-21 12:07:29,453 - root - INFO - Configuration sent to ['ERM864292045293444']
2023-05-21 12:07:29,915 - root - INFO - ['ERM864292045293444'] updated to desired configuration:
2023-05-21 12:07:29,916 - root - INFO - ("{'gnssAlwaysOn': False, 'gnssMaxDuration': 250, 'bluetoothAlwaysOn': True, "
 "'geoLocationRefreshTime': 60}")


184.76950675 185.685429 0.9159222499999942


#### GW Reboot

In [14]:
def get_erm_online():
    if e.get_gateway(GW)['status'] == 'active':
        return True
    return False

print(get_erm_online())
start = time.perf_counter()
e.send_action_to_gateway(GW, GatewayAction.REBOOT_GW)
while not(get_erm_online()):
    time.sleep(2)
end = time.perf_counter()
print(start, end, (end-start))

## Check for MQTT Connection event


True
560.151477875 561.102792167 0.9513142920000064


#### GW OTA

In [20]:
start = time.perf_counter()
fw.firmware_update([GW], desired_version='erm_3.10.33', update_to_latest=False, update_all_connected_bridges=False)
end = time.perf_counter()
print(start, end, (end-start))
start = time.perf_counter()
fw.firmware_update([GW], desired_version='erm_3.10.34', update_to_latest=False, update_all_connected_bridges=False)
end = time.perf_counter()
print(start, end, (end-start))


2023-05-21 12:24:14,588 - root - INFO - Gateway ERM864292045293444 is online.
2023-05-21 12:24:16,018 - root - INFO - No need to update GW ERM864292045293444 to version erm_3.10.33, GW already updated


1188.289997334 1191.164777542 2.8747802079999474


2023-05-21 12:24:17,409 - root - INFO - Gateway ERM864292045293444 is online.
2023-05-21 12:24:18,779 - root - INFO - Will Update GWs ['ERM864292045293444'] from type GatewayType.LTE to version erm_3.10.34
2023-05-21 12:24:19,307 - root - INFO - Configuration sent to ['ERM864292045293444']
2023-05-21 12:24:19,756 - root - INFO - ['ERM864292045293444'] updated to desired configuration:
2023-05-21 12:24:19,757 - root - INFO - '{}'
2023-05-21 12:24:20,194 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:24:30,655 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:24:41,102 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:24:51,545 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:25:01,982 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:25:12,418 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:25:22,867 - root - INFO - Waiting for GW to get back online...
2023-05-21 12:25:33,356 - ro

1191.1650395 1331.059051042 139.89401154200004


#### GW Info

In [22]:
start = time.perf_counter()
print(e.get_gateway_info(GW))

end = time.perf_counter()
print(start, end, (end-start))


{'Cache': 'Pixels=9/128, Meta=5/256', 'Ignition switch state': '0', 'Battery voltage': '3.6', 'Cellular signal': '53A3:00A8AC15@-80 (9460)', 'ICCID': '89972010122060332131', 'Time since last boot': '0:00:00:40', 'ERM firmware version': '3.10.34', 'Cellular network state': 'Home', 'Hardware name': 'StarLink-W1 GWIFI2e32 AG', 'IMEI': '864292045293444', 'Cellular MNC-MCC': '425001', 'Cellular network': 'Partner', 'Main voltage': '12.2', 'APN': 'm2m'}
1412.687417959 1413.126360792 0.43894283300005554


#### Gateway Logs  

In [24]:
start = time.perf_counter()
print(e.get_gateway_logs(GW))
end = time.perf_counter()
print(start, end, (end-start))


None
1478.546560959 1481.433913875 2.887352915999827


#### Bridge Blink

In [20]:
from wiliot_deployment_tools.api.extended_api import BridgeThroughGatewayAction
for brg in BRGS:
    start = time.perf_counter()
    e.send_action_to_bridge(brg, BridgeAction.BLINK_LED)
    end = time.perf_counter()
    print(brg, start, end, (end-start))


7BA7DD8E304A 5294.107723833 5294.628696083 0.5209722500003409
0272A2B3B3E6 5294.628868667 5295.14061475 0.5117460830006166
639186EF4881 5295.140777583 5295.664535458 0.523757874999319
E465A89A65BE 5295.664723667 5296.183167708 0.5184440410002935


#### Bridge Reboot

In [ ]:
from wiliot_deployment_tools.api.extended_api import BridgeThroughGatewayAction
for brg in BRGS:
    e.send_bridge_action_through_gw(GW, BridgeThroughGatewayAction.REBOOT)

#### Power Management

In [ ]:
for brg in BRGS:
    start = time.perf_counter()
    pm.enter_power_mgmt(1, 30, gateway_id=GW, bridge_id=brg)
    end = time.perf_counter()
    print(start, end, (end-start))
    start = time.perf_counter()
    pm.exit_power_mgmt(gateway_id=GW, bridge_id=brg)
    end = time.perf_counter()
    print(start, end, (end-start))

 #### Change Bridge Configuration

In [36]:
for brg in BRGS:
    config = e.get_bridge(brg)['config']
    config.pop('pacingMode')
    new_config = {'txPeriodMs': 5, 'rxTxPeriodMs': 15, 'txRepetition': 0, 'energyPattern': 63, 'pacerInterval': 15, 'txProbability': 50, 'sub1GhzFrequency': 919100, '2.4GhzOutputPower': 2, 'globalPacingGroup': 0, 'otaUpgradeEnabled': 0, 'sub1GhzOutputPower': 32}

    start = time.perf_counter()
    e.change_brg_config()
    end = time.perf_counter()
    print(brg, start, end, (end-start))
    print(config)

{'txPeriodMs': 5, 'rxTxPeriodMs': 15, 'txRepetition': 0, 'energyPattern': 63, 'pacerInterval': 15, 'txProbability': 50, 'sub1GhzFrequency': 919100, '2.4GhzOutputPower': 2, 'globalPacingGroup': 0, 'otaUpgradeEnabled': 0, 'sub1GhzOutputPower': 32}
{'txPeriodMs': 8, 'rxTxPeriodMs': 24, 'txRepetition': 0, 'energyPattern': 50, 'pacerInterval': 15, 'txProbability': 50, 'sub1GhzFrequency': 919100, '2.4GhzOutputPower': 2, 'globalPacingGroup': 1, 'otaUpgradeEnabled': 0, 'sub1GhzOutputPower': 32}
{'txPeriodMs': 9, 'rxTxPeriodMs': 22, 'txRepetition': 0, 'energyPattern': 36, 'pacerInterval': 60, 'txProbability': 50, 'sub1GhzFrequency': 919100, '2.4GhzOutputPower': 2, 'globalPacingGroup': 2, 'otaUpgradeEnabled': 0, 'sub1GhzOutputPower': 32}
{'txPeriodMs': 5, 'rxTxPeriodMs': 15, 'txRepetition': 0, 'energyPattern': 36, 'pacerInterval': 5, 'txProbability': 30, 'sub1GhzFrequency': 919100, '2.4GhzOutputPower': 2, 'globalPacingGroup': 0, 'otaUpgradeEnabled': 0, 'sub1GhzOutputPower': 32}
